In [2]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
df = pd.read_csv('data-1649678101298.csv', sep=',')
df.dropna(axis=0, how='any', inplace=True)

df

,osm_id,date_time,link_dir,speed_kmph,n_time,hours,mins,n_day,stores,n_month,highway,road_lt_m,bus_stop
1,13769164,2018-01-01 00:00:00,1,25,00:00,0,0,MONDAY,CLOSED,JANUARY,primary,72.0,no
2,13769164,2018-01-03 14:45:00,1,21,14:45,14,45,WEDNESDAY,OPEN,JANUARY,primary,72.0,no
3,174019380,2018-01-01 00:00:00,1,31,00:00,0,0,MONDAY,CLOSED,JANUARY,primary,42.0,no
4,176665188,2018-01-01 00:00:00,1,26,00:00,0,0,MONDAY,CLOSED,JANUARY,primary,542.0,yes
5,13769164,2018-01-01 00:15:00,1,22,00:15,0,15,MONDAY,CLOSED,JANUARY,primary,72.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42688,176665188,2018-04-30 23:45:00,1,28,23:45,23,45,MONDAY,CLOSED,APRIL,primary,542.0,yes
42689,174019380,2018-04-30 23:45:00,1,19,23:45,23,45,MONDAY,CLOSED,APRIL,primary,42.0,no
42690,174019380,2018-05-01 00:00:00,1,19,00:00,0,0,TUESDAY,CLOSED,MAY,primary,42.0,no
42691,13769164,2018-05-01 00:00:00,1,19,00:00,0,0,TUESDAY,CLOSED,MAY,primary,72.0,no


In [4]:
df.info()
df.describe()
# df.value_counts('link_dir')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31935 entries, 1 to 42693
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   osm_id      31935 non-null  int64  
 1   date_time   31935 non-null  object 
 2   link_dir    31935 non-null  int64  
 3   speed_kmph  31935 non-null  int64  
 4   n_time      31935 non-null  object 
 5   hours       31935 non-null  int64  
 6   mins        31935 non-null  int64  
 7   n_day       31935 non-null  object 
 8   stores      31935 non-null  object 
 9   n_month     31935 non-null  object 
 10  highway     31935 non-null  object 
 11  road_lt_m   31935 non-null  float64
 12  bus_stop    31935 non-null  object 
dtypes: float64(1), int64(5), object(7)
memory usage: 3.4+ MB


,osm_id,link_dir,speed_kmph,hours,mins,road_lt_m
count,3.193500e+04,31935.0,31935.000000,31935.000000,31935.000000,31935.000000
mean,1.235001e+08,1.0,23.364710,11.589165,22.516674,222.708940
std,7.543989e+07,0.0,8.858938,6.939850,16.777275,230.598565
min,1.376916e+07,1.0,1.000000,0.000000,0.000000,42.000000
25%,1.376916e+07,1.0,17.000000,6.000000,15.000000,42.000000
50%,1.740194e+08,1.0,23.000000,12.000000,30.000000,72.000000
75%,1.766652e+08,1.0,28.000000,18.000000,45.000000,542.000000
max,1.766652e+08,1.0,77.000000,23.000000,45.000000,542.000000


In [5]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

ohe.fit_transform(df[['n_day']])

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [6]:
from sklearn.compose import make_column_transformer

# Continue this by selecting appropriate pre-processing step to each column and apply this to X data frame
column_trans = make_column_transformer(
    (OneHotEncoder(), ['n_day', 'stores', 'n_month', 'bus_stop']),
    remainder='passthrough'
)

column_trans.fit_transform(df)

array([[0.0, 1.0, 0.0, ..., 0, 'primary', 72.0],
       [0.0, 0.0, 0.0, ..., 45, 'primary', 72.0],
       [0.0, 1.0, 0.0, ..., 0, 'primary', 42.0],
       ...,
       [0.0, 0.0, 0.0, ..., 0, 'primary', 42.0],
       [0.0, 0.0, 0.0, ..., 0, 'primary', 72.0],
       [0.0, 0.0, 0.0, ..., 0, 'primary', 542.0]], dtype=object)

# Preprocessing 

In [7]:
# import label encoder to convert categorical data into numerical value
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

df = df.copy()

# Split df into X and y
# selecting the prediction target (label)
y = df.speed_kmph
    
# selecting the 'features'
data_features = ['osm_id', 'hours', 'mins', 'n_day', 'n_month', 'stores', 'road_lt_m', 'bus_stop']
# data_features = ['hours', 'mins', 'n_day', 'n_month', 'stores']
X = df[data_features]

# Continue this by selecting appropriate pre-processing step to each column and apply this to X data frame
# Emit hghway and bus_stop for one segment process ['n_day', 'stores', 'n_month', 'highway', 'bus_stop']
column_trans = make_column_transformer(
    (OneHotEncoder(), ['n_day', 'stores', 'n_month', 'bus_stop']),
    remainder='passthrough'
)

X = column_trans.fit_transform(X)

# Train-test split
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

# Scale X
print('Variance before scaler:', train_X.var(), sep='\n')
print('\n')
scaler = StandardScaler()
scaler.fit(train_X)
train_X = pd.DataFrame(scaler.transform(train_X))
val_X = pd.DataFrame(scaler.transform(val_X))

print('Variance after scaler:', train_X.var(), sep='\n')
print('\n')

print('Training Data', train_X, sep='\n')
print('Test Data', val_X, sep='\n')

Variance before scaler:
920443936122033.5


Variance after scaler:
0     1.000045
1     1.000045
2     1.000045
3     1.000045
4     1.000045
5     1.000045
6     1.000045
7     1.000045
8     1.000045
9     1.000045
10    1.000045
11    1.000045
12    1.000045
13    1.000045
14    1.000045
15    1.000045
16    1.000045
17    1.000045
18    1.000045
19    1.000045
20    1.000045
21    1.000045
dtype: float64


Training Data
             0         1         2         3         4         5         6   \
0     -0.403956 -0.418602 -0.409483 -0.401255  2.482428 -0.402156 -0.419266   
1     -0.403956 -0.418602 -0.409483 -0.401255  2.482428 -0.402156 -0.419266   
2     -0.403956 -0.418602 -0.409483 -0.401255  2.482428 -0.402156 -0.419266   
3     -0.403956 -0.418602  2.442102 -0.401255 -0.402831 -0.402156 -0.419266   
4     -0.403956 -0.418602 -0.409483  2.492181 -0.402831 -0.402156 -0.419266   
...         ...       ...       ...       ...       ...       ...       ...   
22349  2.475518 -0.

In [8]:
X

array([[  0.,   1.,   0., ...,   0.,   0.,  72.],
       [  0.,   0.,   0., ...,  14.,  45.,  72.],
       [  0.,   1.,   0., ...,   0.,   0.,  42.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,  42.],
       [  0.,   0.,   0., ...,   0.,   0.,  72.],
       [  0.,   0.,   0., ...,   0.,   0., 542.]])

# Training/Model Selection

In [21]:
models = {
    "                     Linear Regression": LinearRegression(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(),
    "                         Random Forest": RandomForestRegressor(),
    "                     Gradient Boosting": GradientBoostingRegressor()
}

for name, model in models.items():
    model.fit(train_X, train_y)
    print(name + " trained.")

                     Linear Regression trained.
                   K-Nearest Neighbors trained.
                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.


# Results

Ref: https://www.kaggle.com/getting-started/27261
Every estimator or model in Scikit-learn has a score method after being trained on the data, usually X_train, y_train.

When you call score on classifiers like LogisticRegression, RandomForestClassifier, etc. the method computes the accuracy score by default (accuracy is #correct_preds / #all_preds). By default, the score method does not need the actual predictions.

##### Score method of regressors
When score is called on regressors, the coefficient of determination - R2 is calculated by default. As in classifiers, the score method is simply a shorthand to calculate R2 since it is commonly used to assess the performance of a regressor.

More on model evaluation: https://coderzcolumn.com/tutorials/machine-learning/model-evaluation-scoring-metrics-scikit-learn-sklearn

In [22]:
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(val_X, val_y)))
    print(name + " R^2 Score: {:.5f}".format(model.score(train_X, train_y)))

                     Linear Regression R^2 Score: 0.12324
                     Linear Regression R^2 Score: 0.14039
                   K-Nearest Neighbors R^2 Score: 0.34308
                   K-Nearest Neighbors R^2 Score: 0.56507
                        Neural Network R^2 Score: 0.42782
                        Neural Network R^2 Score: 0.46669
Support Vector Machine (Linear Kernel) R^2 Score: 0.11694
Support Vector Machine (Linear Kernel) R^2 Score: 0.13620
   Support Vector Machine (RBF Kernel) R^2 Score: 0.35539
   Support Vector Machine (RBF Kernel) R^2 Score: 0.38431
                         Decision Tree R^2 Score: 0.16641
                         Decision Tree R^2 Score: 0.65624
                         Random Forest R^2 Score: 0.27309
                         Random Forest R^2 Score: 0.64945
                     Gradient Boosting R^2 Score: 0.40249
                     Gradient Boosting R^2 Score: 0.41918


In [23]:
results = """
                     Linear Regression R^2 Score: 0.10938
                   K-Nearest Neighbors R^2 Score: 0.33238
                        Neural Network R^2 Score: -14.07604
Support Vector Machine (Linear Kernel) R^2 Score: -6.82171
   Support Vector Machine (RBF Kernel) R^2 Score: 0.04857
                         Decision Tree R^2 Score: 0.15332
                         Random Forest R^2 Score: 0.26139
                     Gradient Boosting R^2 Score: 0.38760
"""
print('Results before scalling: ', results)

Results before scalling:  
                     Linear Regression R^2 Score: 0.10938
                   K-Nearest Neighbors R^2 Score: 0.33238
                        Neural Network R^2 Score: -14.07604
Support Vector Machine (Linear Kernel) R^2 Score: -6.82171
   Support Vector Machine (RBF Kernel) R^2 Score: 0.04857
                         Decision Tree R^2 Score: 0.15332
                         Random Forest R^2 Score: 0.26139
                     Gradient Boosting R^2 Score: 0.38760



In [24]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

for name, model in models.items():
    val_predict = model.predict(val_X)
    print(name + " R^2 Score: {:.5f}".format(r2_score(val_y, val_predict)))
    print(name + " RMSE: {:.5f}".format(np.sqrt(mean_squared_error(val_y, val_predict))))
    print(name + " MAE: {:.5f}".format(mean_absolute_error(val_y, val_predict)))

                     Linear Regression R^2 Score: 0.12324
                     Linear Regression RMSE: 8.34117
                     Linear Regression MAE: 6.56825
                   K-Nearest Neighbors R^2 Score: 0.34308
                   K-Nearest Neighbors RMSE: 7.22009
                   K-Nearest Neighbors MAE: 5.39831
                        Neural Network R^2 Score: 0.42782
                        Neural Network RMSE: 6.73833
                        Neural Network MAE: 5.05970
Support Vector Machine (Linear Kernel) R^2 Score: 0.11694
Support Vector Machine (Linear Kernel) RMSE: 8.37108
Support Vector Machine (Linear Kernel) MAE: 6.55972
   Support Vector Machine (RBF Kernel) R^2 Score: 0.35539
   Support Vector Machine (RBF Kernel) RMSE: 7.15212
   Support Vector Machine (RBF Kernel) MAE: 5.39387
                         Decision Tree R^2 Score: 0.16641
                         Decision Tree RMSE: 8.13320
                         Decision Tree MAE: 6.05900
                      